## Bibliotecas

In [ ]:
import pandas as pd
import ast

## Importadores

In [ ]:
# Lista de importadores de capacete de moto
importadorMot = pd.read_excel('https://raw.githubusercontent.com/FredericoCS1/Importacoes/main/Importadores/importadoresMoto.xlsx')
importadorMot.head()

In [ ]:
importadorMot = importadorMot['ImportadoresMoto'].to_list()

In [ ]:
# Lista de importadores adicionais
importadorExt = pd.read_excel('https://raw.githubusercontent.com/FredericoCS1/Importacoes/main/Importadores/importadoresExtra.xlsx')
importadorExt.head()

In [ ]:
importadorExt = importadorExt['ImportadoresExtra'].to_list()

In [ ]:
# Lista de importadores de Capacete de Bicicleta
importadorBic = pd.read_excel('https://raw.githubusercontent.com/FredericoCS1/Importacoes/main/Importadores/importadoresBike.xlsx')
importadorBic.head()

In [ ]:
importadorBic = importadorBic['ImportadoresBike'].to_list()

## Código

In [ ]:
# Leitura da base, retornando os primeiros valores
df = pd.read_excel('Relatório de Importação.xlsx')
df.head()

In [ ]:
# Retorna a quantidade de valores de uma coluna da base
df['Palavras chave de descrição do produto'].count()

In [ ]:
# Tirando a primeira linha da base, porque só contem valores nulos e sem importância
df.drop([0, 0], inplace=True)

In [ ]:
# Criando uma cópia da base para estudo, assim não afeta a principal
dfCap = df.copy()

In [ ]:
# Dentro da cópia chamada de 'dfCap', foi criada uma variável, 'filterCap', esta irá armazenar as modificações feitas na base cópia
# Esta parte faz uma busca dentro da coluna, 'Palavras chave de descrição do produto', para os valores dentro de 'contains', todos os valores da base são considerados string
filterCap = dfCap[
    dfCap['Palavras chave de descrição do produto']
    .astype(str)
    .str.contains(r'CAPACETE|HELMET', na=False)
# Esta segunda parte faz uma busca na mesma coluna, mas agora retirando os valores dentro de 'contains', valores para string
    & ~dfCap['Palavras chave de descrição do produto']
    .astype(str)
    .str.contains(r'PROTETOR|BOMBEIRO|INCENDIO|SKI|INDUSTRIAL|'
                  r'EQUITAÇÃO|KART|PASSAGEIROS|MASCARA|EPI|'
                  r'AUTOGRAFADO|HIPISMO|OUTSHOCK|AUTOMOBILISMO|'
                  r'WEDZE', na=False)
]
# Para a criação de uma novo DF, tabela, será retirado os index, para que não afetem a leitura
filterCap.reset_index(drop=True, inplace=True)
# Retorna a quantidade de valores de uma coluna da base
filterCap['Palavras chave de descrição do produto'].count()

In [ ]:
# Transformação do DF em excel
filterCap.to_excel('Relatório de Importação de Capacetes Ajustado.xlsx', index=False)

In [ ]:
# Leitura da nova base, retornando a quantidade de valores
df2 = pd.read_excel('Relatório de Importação de Capacetes Ajustado.xlsx')
df2['Palavras chave de descrição do produto'].count()

In [ ]:
# Cópia da base
dfFim = df2.copy()

In [ ]:
# A filtragem desta parte consiste na busca dos valores dentro da lista 'importadorMot'
# Aplica uma função lambda, para cada elemento 'x' da coluna, 'PROVÁVEL IMPORTADOR'
# 'x' representa um valor único da coluna, passado na função lammbda
# Verifica se existe algum valor, em string, dentro da lista 'importadorMot' que é subvalor, substring, de 'x'
# Verifica se existe algum elemento, 'imp' dentro de 'importadorMot', que esta contido em 'x'
filterFim = dfFim[dfFim['PROVÁVEL IMPORTADOR']
    .astype(str)
    .apply(lambda x: any(imp in x for imp in importadorBic))
]
# Retira index
filterFim.reset_index(drop=True, inplace=True)
# Quantidade de valores
filterFim['Palavras chave de descrição do produto'].count()

In [ ]:
# Transformação do DF em excel
filterFim.to_excel('Relatório de Importação de Capacetes de Bike.xlsx', index=False)

In [ ]:
df3 = pd.read_excel('Relatório de Importação de Capacetes de Bike.xlsx')
df3['Palavras chave de descrição do produto'].count()

In [ ]:
# Cópia da base
dfRes = df3.copy()

In [ ]:
# Lista vazia para os dados extraídos
extracted_data = []

# Função for que itera para cada linha do DataFrame
for index, row in dfRes.iterrows():
  # Tenta executar as funções
  try:
    # Transforma a string em um dicionário Python
    row_dict = ast.literal_eval(row['Relevância por palavra-chave'])
    # Função for que itera cada parâmetro do dicionário
    for key, inner_dict in row_dict.items():
      # Para cada parâmetro do dicionário, um dado é extraído e colocado na lista
        extracted_data.append({
        'row_index': index,
        'Keywords': inner_dict['keywords'],
        'Quantidade Estatística': inner_dict['relevancia_qtd'],
        'Valor FOB Unitário': inner_dict['relevancia_fob']
    })
  # Se tiver um erro, separa dos extraídos para que não pare a execução
  except (ValueError, KeyError, SyntaxError) as e:
    print(f"Erro ao processar a linha {index}: {e}")

# Transforma a lista dos extraídos em um DataFrame
dfRes_extracted = pd.DataFrame(extracted_data)

# Transforma os valores para numérico
dfRes_extracted['Quantidade Estatística'] = pd.to_numeric(dfRes_extracted['Quantidade Estatística'], errors='coerce')
dfRes_extracted['Valor FOB Unitário'] = pd.to_numeric(dfRes_extracted['Valor FOB Unitário'], errors='coerce')

# Une esse novo DF com o resto da base
dfRes_final = dfRes.merge(dfRes_extracted, left_index=True, right_on = 'row_index', how='left')
# Excluí colunas indesejadas
dfRes_final.drop(columns=['Relevância por palavra-chave', 'row_index'], inplace=True)

dfRes_final.head()

In [ ]:
# Divide a coluna FOB pela Quantidade estatística
dfRes_final['QTD por FOB'] = dfRes_final['Valor FOB Unitário'] / dfRes_final['Quantidade Estatística']
dfRes_final.head()

In [ ]:
# DF para Excel
dfRes_final.to_excel('Relatório de Importação de Capacetes de Bike Final.xlsx', index=False)

In [ ]:
df4 = pd.read_excel('Relatório de Importação de Capacetes de Bike Final.xlsx')
df4.head()

In [ ]:
dfExt = df4.copy()

In [ ]:
# Lista contendo marcas de capacete de moto
brandsMot = [
    "AGV", "AIROH", "ALPINESTAR", "ASW", "ASTONE", "AXXIS", "BIEFFE", "BMW", "GP TECH",
    "HELT", "HJC", "KYT", "LS2", "LUCCA", "NEXX", "NOLAN", "NORISK", "NZI", "PEELS",
    'RACE', "RACE TECH", "SCHUBERTH", "SHARK", "SHOEI", "SMK", "STG", "SUOMY", "TEXX",
    "X11", 'Reaper']

In [ ]:
# Lista contendo marcas de capacete de bicicleta
bandsBic = [
    'ARAI', 'BELL', 'ABUS']

In [ ]:
# Vai trazer todos os valores que não estão contidos dentro da lista 'brandsMot'
filterExt = dfExt[
    ~dfExt['Palavras chave de descrição do produto']
    .astype(str)
    .str.contains('|'.join(brandsMot))]
# Retira index
filterExt.reset_index(drop=True, inplace=True)
# Quantidade de valores
filterExt['Palavras chave de descrição do produto'].count()